## Here we compare models of different depth and architecture on clear data.

In [1]:
import warnings
warnings.filterwarnings("ignore")

import matplotlib as mpl
mpl.use("TkAgg")

from sklearn.preprocessing import normalize#, MinMaxScaler
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import h5py
import mne
import keras
import keras.backend as K
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Reshape, GRU, LSTM, Lambda, RepeatVector, Reshape, Dropout, Conv1D, UpSampling1D, Bidirectional

from os import walk, listdir


%matplotlib inline
data_path = "../../../data/train/"
clear_data_path = "/media/valbub/Docs/data/train/"
raw_data_path = "../../../data/resting_state/"

Using TensorFlow backend.


In [2]:
class AutoEncoder(object):
    def __init__(self, 
             input_dim = (7, 58), 
             encoded_dim = (1, 58), 
             loss="mse", 
             optimizer="adadelta", 
             activation=("relu", "sigmoid", "tanh", "elu"),
             act_idx=(0, 0),
             kernel = 7):
        
            self.input_dim = input_dim
            self.encoded_dim = encoded_dim
            
            class MinMaxScaler():

                def __init__(self, minimum=None, maximum=None):
                    self.minimum = minimum
                    self.maximum = maximum

                def fit_transform(self, X):
                    if self.minimum is None or self.maximum is None:
                        self.minimum = np.min(X, axis=(0, 1))
                        self.maximum = np.max(X, axis=(0, 1))
                    return (X - self.minimum) / (self.maximum - self.minimum)

                def transform(self, X):
                    return (np.array(X) - self.minimum) / (self.maximum - self.minimum)

                def reverse_transform(self, X_scl):
                    return X_scl * (self.maximum - self.minimum) + self.minimum

            self.scaler = MinMaxScaler()
    
    def prepare_clear_data(self, data_path, limit=2):
        train_eeg_dir = data_path
        train_eeg_names = [x for x in listdir(train_eeg_dir) 
                         if x[-3:] == ".h5"]
        data = np.zeros((0, self.input_dim[0], self.input_dim[1]))

        flag = 0
        err_files = 0
        for eeg_name in train_eeg_names:
            if flag == limit:
                break
            flag += 1
            h5_file = h5py.File(train_eeg_dir + eeg_name, 'r')
            a_group_key = list(h5_file.keys())[0]
            eeg_data = np.array(h5_file[a_group_key]).T
            batches = np.array(self._getBatches(eeg_data, batch_size=self.input_dim[0]))
            data = np.concatenate((data, batches), axis=0)
        return data
    
    def prepare_raw_data(self, data_path, limit=2):
        def preparefile(file_path):
            raw = np.array(mne.io.read_raw_brainvision(file_path, preload=True).to_data_frame())
            batches = np.array(self._getBatches(raw, batch_size=self.input_dim[0]))
            del raw
            return batches
        files = []
        data = []
        for elem in walk(data_path):
            for file in elem[-1]:
                if file[-4:] == "vhdr":
                    files.append(file)
        data = np.ndarray(shape=(0, self.input_dim[0], self.input_dim[1]))
        flag = 0
        for file in files:
            flag += 1
            file_name = data_path + file
            if flag == limit:
                break
            batches =  preparefile(file_name)
            data = np.concatenate((data, batches), axis=0)
        return data
    
    def fit(self, X_train, epochs=50):
        X_scaled = self.scaler.fit_transform(X_train)
        return self.autoencoder.fit(X_scaled, X_scaled, epochs = epochs)
    
    def fit_scaler(self, X_train):
        self.scaler.fit_transform(X_train)
    
    def encode(self, df):
        return self._predict(df, self.encoder, self.input_dim[0])
    
    def decode(self, df):
        return self._predict(df, self.decoder, self.encoded_dim[1])
    
    def run(self, df):
        return self._predict(df, self.autoencoder, self.input_dim[0])
    
    def save(self, path, part="autoencoder"):
        if part == "encoder":
            self.encoder.save(path)
        elif part == "decoder":
            self.decoder.save(path)
        elif part == "autoencoder":
            self.autoencoder.save(path)
        pass
    
    def load(self, path, part="autoencoder", X_train=None):
        if part == "encoder":
            self.encoder = keras.models.load_model(path)
        elif part == "decoder":
            self.decoder = keras.models.load_model(path)
        elif part == "autoencoder":
            self.autoencoder = keras.models.load_model(path)
        if X_train is not None:
            self.fit_scaler(X_train)
    

    def _predict(self, df, model):
        batches = self.scaler.transform(df)
        batches = tuple(self._predictBatch(batch.reshape((1, *batch.shape)), model) for batch in batches)
        batches = self._concatBatches(batches) 
        return self.scaler.reverse_transform(batches)
    
    def _predictBatch(self, batch, model):
        return model.predict(batch)
    
    def _getBatches(self, arr, batch_size, axis=0):
        n_batches = arr.shape[axis] // batch_size
        result = np.array_split(arr, n_batches, axis=axis)
        i = 0
        while result[i].shape[0] != batch_size:
            i += 1
        result = result[i:]
        return result
    
    def _concatBatches(self, batches, axis=0):
        return np.concatenate(batches, axis=axis)


In [3]:
class AU_Stage_1(AutoEncoder):
        def __init__(self, 
                 input_dim = (7, 58), 
                 encoded_dim = (1, 58), 
                 loss="mse", 
                 optimizer="adadelta", 
                 activation=("elu", "sigmoid"),
                 kernel = 7):
        
            self.input_dim = input_dim
            self.encoded_dim = encoded_dim

            #Encoder
            self._inputs = Input(shape=input_dim)
            self._flat = Flatten()(self._inputs)
            self._dense = Dense(units=np.prod(encoded_dim), activation=activation[0])(self._flat)
            self._encoded = Reshape(encoded_dim)(self._dense)

            #Decoder
            self._encoded_inputs = Input(shape=(encoded_dim[0]*encoded_dim[1],))
            self._flat_decoded = Dense(input_dim[0]*input_dim[1], activation=activation[1])(self._encoded_inputs)
            self._decoded = Reshape(input_dim)(self._flat_decoded)

            #Models
            self.encoder = Model(self._inputs, self._encoded)
            self.decoder = Model(self._encoded_inputs, self._decoded)
            self.autoencoder = Model(self._inputs, self.decoder(self.encoder(self._inputs)))
            
            self.autoencoder.compile(optimizer=optimizer, loss=loss)

            class MinMaxScaler():

                def __init__(self, minimum=None, maximum=None):
                    self.minimum = minimum
                    self.maximum = maximum

                def fit_transform(self, X):
                    if self.minimum is None or self.maximum is None:
                        self.minimum = np.min(X, axis=(0, 1))
                        self.maximum = np.max(X, axis=(0, 1))
                    return (X - self.minimum) / (self.maximum - self.minimum)

                def transform(self, X):
                    return (np.array(X) - self.minimum) / (self.maximum - self.minimum)

                def reverse_transform(self, X_scl):
                    return X_scl * (self.maximum - self.minimum) + self.minimum

            self.scaler = MinMaxScaler()

In [4]:
class AU_Stage_2(AutoEncoder):
        def __init__(self, 
                 input_dim = (7, 58), 
                 encoded_dim = (1, 58), 
                 loss="mse", 
                 optimizer="adadelta", 
                 activation=("elu", "sigmoid"),
                 kernel = 7):
        
            self.input_dim = input_dim
            self.encoded_dim = encoded_dim

            #Encoder
            self._inputs = Input(shape=input_dim)
            self._conv = Conv1D(filters=encoded_dim[1], kernel_size=kernel)(self._inputs) 
            self._dense = Dense(units=np.prod(encoded_dim), activation=activation[0])(self._conv)
            self._encoded = Reshape(encoded_dim)(self._dense)

            #Decoder
            self._encoded_inputs = Input(shape=encoded_dim)
            self._flat_decoded = Dense(units=np.prod(input_dim), activation=activation[1])(self._encoded_inputs)
            self._decoded = Reshape(input_dim)(self._flat_decoded)

            #Models
            self.encoder = Model(self._inputs, self._encoded)
            self.decoder = Model(self._encoded_inputs, self._decoded)
            self.autoencoder = Model(self._inputs, self.decoder(self.encoder(self._inputs)))

            self.autoencoder.compile(optimizer=optimizer, loss=loss)
            
            class MinMaxScaler():

                def __init__(self, minimum=None, maximum=None):
                    self.minimum = minimum
                    self.maximum = maximum

                def fit_transform(self, X):
                    if self.minimum is None or self.maximum is None:
                        self.minimum = np.min(X, axis=(0, 1))
                        self.maximum = np.max(X, axis=(0, 1))
                    return (X - self.minimum) / (self.maximum - self.minimum)

                def transform(self, X):
                    return (X - self.minimum) / (self.maximum - self.minimum)

                def reverse_transform(self, X_scl):
                    return X_scl * (self.maximum - self.minimum) + self.minimum

            self.scaler = MinMaxScaler()


In [5]:
class AU_Stage_3(AutoEncoder):
        def __init__(self, 
                 input_dim = (7, 58), 
                 encoded_dim = (1, 58), 
                 loss="mse", 
                 optimizer="adadelta", 
                 activation=("elu", "sigmoid"),
                 kernel = 7):
        
            self.input_dim = input_dim
            self.encoded_dim = encoded_dim

            #Encoder
            self._inputs = Input(shape=input_dim)
            self._lstm = LSTM(input_dim[1], return_sequences=True, dropout=0, recurrent_dropout=0.1)(self._inputs)
            self._conv = Conv1D(filters=encoded_dim[1], kernel_size=input_dim[0])(self._lstm)
            self._dense = Dense(units=np.prod(encoded_dim), activation=activation[0])(self._conv)
            self._encoded = Reshape(encoded_dim)(self._dense)
            
            #Decoder
            self._encoded_inputs = Input(shape=encoded_dim)
            self._flat_decoded_1 = Dense(units=np.prod(input_dim), activation=activation[1])(self._encoded_inputs)
            self._flat_decoded_2 = UpSampling1D(size=input_dim[0] // encoded_dim[0])(self._flat_decoded_1)
            self._lstm_2 = LSTM(input_dim[1], return_sequences=True)(self._flat_decoded_2)
            self._decoded = Reshape(input_dim)(self._lstm_2)
            
            #Models
            self.encoder = Model(self._inputs, self._encoded)
            self.decoder = Model(self._encoded_inputs, self._decoded)
            self.autoencoder = Model(self._inputs, self.decoder(self.encoder(self._inputs)))
            
            self.autoencoder.compile(optimizer=optimizer, loss=loss)
            
            class MinMaxScaler():

                def __init__(self, minimum=None, maximum=None):
                    self.minimum = minimum
                    self.maximum = maximum

                def fit_transform(self, X):
                    if self.minimum is None or self.maximum is None:
                        self.minimum = np.min(X, axis=(0, 1))
                        self.maximum = np.max(X, axis=(0, 1))  + 1e-10
                    return (X - self.minimum) / (self.maximum - self.minimum)

                def transform(self, X):
                    return (X - self.minimum) / (self.maximum - self.minimum)

                def reverse_transform(self, X_scl):
                    return X_scl * (self.maximum - self.minimum) + self.minimum

            self.scaler = MinMaxScaler()

In [22]:
class AU_Stage_3_5(AutoEncoder):
        def __init__(self, 
                 input_dim = (7, 58), 
                 encoded_dim = (1, 58), 
                 loss="mse", 
                 optimizer="adadelta", 
                 activation=("elu", "sigmoid"),
                 kernel = 7):
        
            self.input_dim = input_dim
            self.encoded_dim = encoded_dim

            #Encoder
            self._inputs = Input(shape=input_dim)
            self._gru = GRU(input_dim[1], return_sequences=True, dropout=0, recurrent_dropout=0.1)(self._inputs)
            self._conv = Conv1D(filters=encoded_dim[1], kernel_size=input_dim[0])(self._gru)
            self._dense = Dense(units=np.prod(encoded_dim), activation=activation[0])(self._conv)
            self._encoded = Reshape(encoded_dim)(self._dense)

            #Decoder
            self._encoded_inputs = Input(shape=encoded_dim)
            self._flat_decoded_1 = Dense(units=np.prod(input_dim), activation=activation[1])(self._encoded_inputs)
            self._flat_decoded_2 = UpSampling1D(size=input_dim[0] // encoded_dim[0])(self._flat_decoded_1)
            self._gru_2 = GRU(input_dim[1], return_sequences=True)(self._flat_decoded_2)
            self._decoded = Reshape(input_dim)(self._gru_2)

            #Models
            self.encoder = Model(self._inputs, self._encoded)
            self.decoder = Model(self._encoded_inputs, self._decoded)
            self.autoencoder = Model(self._inputs, self.decoder(self.encoder(self._inputs)))

            self.autoencoder.compile(optimizer=optimizer, loss=loss)

            class MinMaxScaler():

                def __init__(self, minimum=None, maximum=None):
                    self.minimum = minimum
                    self.maximum = maximum

                def fit_transform(self, X):
                    if self.minimum is None or self.maximum is None:
                        self.minimum = np.min(X, axis=(0, 1))
                        self.maximum = np.max(X, axis=(0, 1)) + 1e-10
                    return (X - self.minimum) / (self.maximum - self.minimum)

                def transform(self, X):
                    return (X - self.minimum) / (self.maximum - self.minimum)

                def reverse_transform(self, X_scl):
                    return X_scl * (self.maximum - self.minimum) + self.minimum

            self.scaler = MinMaxScaler()

In [7]:
class AU_Stage_4(AutoEncoder):
        def __init__(self, 
                 input_dim = (7, 58), 
                 encoded_dim = (1, 58), 
                 loss="mse", 
                 optimizer="adadelta", 
                 activation=("elu", "sigmoid"),
                 kernel = 7):
        
            self.input_dim = input_dim
            self.encoded_dim = encoded_dim

            #Encoder
            self._inputs = Input(shape=input_dim)
            self._lstm = LSTM(input_dim[1], return_sequences=True, dropout=0, recurrent_dropout=0.1)(self._inputs)
            self._reshape = Reshape((-1, input_dim[1] * input_dim[0]))(self._lstm)
            self._dense = Dense(units=np.prod(encoded_dim), activation=activation[0])(self._reshape)
            self._encoded = Reshape(encoded_dim)(self._dense)

            #Decoder
            self._encoded_inputs = Input(shape=encoded_dim)
            self._flat_decoded_1 = Dense(units=np.prod(input_dim), activation=activation[1])(self._encoded_inputs)
            self._lstm_2 = LSTM(input_dim[1] * input_dim[0], return_sequences=True)(self._flat_decoded_1)
            self._decoded = Reshape((-1, input_dim[1]))(self._lstm_2)

            #Models
            self.encoder = Model(self._inputs, self._encoded)
            self.decoder = Model(self._encoded_inputs, self._decoded)
            self.autoencoder = Model(self._inputs, self.decoder(self.encoder(self._inputs)))

            self.autoencoder.compile(optimizer=optimizer, loss=loss)

            class MinMaxScaler():

                def __init__(self, minimum=None, maximum=None):
                    self.minimum = minimum
                    self.maximum = maximum

                def fit_transform(self, X):
                    if self.minimum is None or self.maximum is None:
                        self.minimum = np.min(X, axis=(0, 1))
                        self.maximum = np.max(X, axis=(0, 1)) + 1e-10
                    return (X - self.minimum) / (self.maximum - self.minimum)

                def transform(self, X):
                    return (X - self.minimum) / (self.maximum - self.minimum)

                def reverse_transform(self, X_scl):
                    return X_scl * (self.maximum - self.minimum) + self.minimum

            self.scaler = MinMaxScaler()

In [8]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

epoch_numb = 50
limit = 14

In [9]:
# here we get data for all experiments in future
au_1 = AU_Stage_1()
data_set = au_1.prepare_clear_data(clear_data_path, limit=limit)
train_data, test_data = train_test_split(data_set, random_state=0, test_size=0.3)
data_set.shape

(1188300, 7, 58)

In [10]:
au_1.load("../experiment-1_batch7/model_stage_1/au", X_train=train_data)

In [11]:
pr = au_1._predict(test_data, au_1.autoencoder)
ds = np.concatenate(test_data)
p = np.concatenate(pr)
print('\nDense')
print('\nClear data')
print(r2_score(ds, p))
print(mean_absolute_error(ds, p))
print(mean_squared_error(ds, p))
print()


Dense

Clear data
0.8247301200656326
4.832945938199143e-06
7.168427148160001e-09



In [12]:
del au_1

In [13]:
au_2 = AU_Stage_2()
# here we assume that batches are the same and use data from previous model to compare models correctly
# data_set = au_2.prepare_clear_data(clear_data_path, limit=limit)
# train_data, test_data = train_test_split(data_set, random_state=0, test_size=0.3)

au_2.load("../experiment-1_batch7/model_stage_2/au", X_train=train_data)

In [14]:
pr = au_2._predict(test_data, au_2.autoencoder)
ds = np.concatenate(test_data)
p = np.concatenate(pr)
print('\nDense+Conv')
print('\nClear data')
print(r2_score(ds, p))
print(mean_absolute_error(ds, p))
print(mean_squared_error(ds, p))
print()


Dense+Conv

Clear data
0.8257848227367229
5.193955342016033e-06
8.964024932534042e-09



In [15]:
del au_2

In [16]:
au_3 = AU_Stage_3()
# here we assume that batches are the same and use data from previous model to compare models correctly
# data_set = au_3.prepare_clear_data(clear_data_path, limit=limit)
# train_data, test_data = train_test_split(data_set, random_state=0, test_size=0.3)

au_3.fit(train_data, epochs=epoch_numb)
folder = "./model_stage_3/"
au_3.save(folder + "au")
au_3.save(folder + "en")
au_3.save(folder + "de")

Epoch 1/50
831810/831810 [==============================] - 501s 602us/step - loss: 5.8822e-04
Epoch 2/50
831810/831810 [==============================] - 502s 604us/step - loss: 2.3527e-04
Epoch 3/50
831810/831810 [==============================] - 503s 605us/step - loss: 1.7283e-04
Epoch 4/50
831810/831810 [==============================] - 502s 604us/step - loss: 1.4425e-04
Epoch 5/50
831810/831810 [==============================] - 503s 605us/step - loss: 1.3224e-04
Epoch 6/50
831810/831810 [==============================] - 503s 605us/step - loss: 1.2477e-04
Epoch 7/50
831810/831810 [==============================] - 504s 606us/step - loss: 1.1903e-04
Epoch 8/50
831810/831810 [==============================] - 504s 606us/step - loss: 1.1630e-04
Epoch 9/50
831810/831810 [==============================] - 504s 606us/step - loss: 1.1483e-04
Epoch 10/50
831810/831810 [==============================] - 504s 606us/step - loss: 1.1375e-04
Epoch 11/50
831810/831810 [======================

In [17]:
pr = au_3._predict(test_data, au_3.autoencoder)
ds = np.concatenate(test_data)
p = np.concatenate(pr)
print("\nDense+Conv+LSTM")
print('\nClear data')
print(r2_score(ds, p))
print(mean_absolute_error(ds, p))
print(mean_squared_error(ds, p))
print()


Dense+Conv+LSTM

Clear data
0.7088568095962166
9.072323809230355e-06
1.2973953233257075e-08



In [18]:
del au_3

In [23]:
au_3_5 = AU_Stage_3_5()
# here we assume that batches are the same and use data from previous model to compare models correctly
# data_set = au_3_5.prepare_clear_data(clear_data_path, limit=limit)
# train_data, test_data = train_test_split(data_set, random_state=0, test_size=0.3)

au_3_5.load("../experiment-1_batch7/model_stage_3_5/au", X_train=train_data)

In [24]:
pr = au_3_5._predict(test_data, au_3_5.autoencoder)
ds = np.concatenate(test_data)
p = np.concatenate(pr)
print("\nDense+Conv+GRU")
print('\nClear data')
print(r2_score(ds, p))
print(mean_absolute_error(ds, p))
print(mean_squared_error(ds, p))
print()


Dense+Conv+GRU

Clear data
0.7829797791169523
6.931606360453831e-06
2.0235480838020267e-09



In [25]:
del au_3_5

In [26]:
au_4 = AU_Stage_4()
# here we assume that batches are the same and use data from previous model to compare models correctly
# data_set = au_4.prepare_clear_data(clear_data_path, limit=limit)
# train_data, test_data = train_test_split(data_set, random_state=0, test_size=0.3)

au_4.fit(train_data, epochs=epoch_numb)
folder = "./model_stage_4/"
au_4.save(folder + "au")
au_4.save(folder + "en")
au_4.save(folder + "de")

Epoch 1/50
831810/831810 [==============================] - 572s 688us/step - loss: 7.8191e-04
Epoch 2/50
831810/831810 [==============================] - 530s 637us/step - loss: 3.1442e-04
Epoch 3/50
831810/831810 [==============================] - 525s 631us/step - loss: 2.7627e-04
Epoch 4/50
831810/831810 [==============================] - 517s 622us/step - loss: 2.6624e-04
Epoch 5/50
831810/831810 [==============================] - 516s 621us/step - loss: 2.6080e-04
Epoch 6/50
831810/831810 [==============================] - 514s 618us/step - loss: 2.3560e-04
Epoch 7/50
831810/831810 [==============================] - 514s 618us/step - loss: 1.7652e-04
Epoch 8/50
831810/831810 [==============================] - 514s 619us/step - loss: 1.6668e-04
Epoch 9/50
831810/831810 [==============================] - 515s 619us/step - loss: 1.6344e-04
Epoch 10/50
831810/831810 [==============================] - 515s 619us/step - loss: 1.6121e-04
Epoch 11/50
831810/831810 [======================

In [27]:
pr = au_4._predict(test_data, au_4.autoencoder)
ds = np.concatenate(test_data)
p = np.concatenate(pr)
print("\nDense+LSTM")
print('\nClear data')
print(r2_score(ds, p))
print(mean_absolute_error(ds, p))
print(mean_squared_error(ds, p))
print()


Dense+LSTM

Clear data
0.762404942330928
5.4105242149271395e-06
1.0132845665360928e-08



In [28]:
del au_4